<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkML311Coursera747-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# Machine Learning Foundation

## Course 5, Part g: Transfer Learning DEMO


For this exercise, we will use the well-known MNIST digit data. To illustrate the power and concept of transfer learning, we will train a CNN on just the digits 5,6,7,8,9.  Then we will train just the last layer(s) of the network on the digits 0,1,2,3,4 and see how well the features learned on 5-9 help with classifying 0-4.

Adapted from https://github.com/fchollet/keras/blob/master/examples/mnist_transfer_cnn.py


In [21]:
import datetime
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
#from tensorflow import keras
#from tensorflow.keras.datasets import mnist
#from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
#from tensorflow.keras.layers import Conv2D, MaxPooling2D
#from tensorflow.keras import backend as K

In [22]:
#used to help some of the timing functions
now = datetime.datetime.now

In [23]:
# set some parameters
batch_size = 128
num_classes = 5
epochs = 5

In [24]:
# set some more parameters
img_rows, img_cols = 28, 28
filters = 32
pool_size = 2
kernel_size = 3

In [25]:
## This just handles some variability in how the input data is loaded

if K.image_data_format() == 'channels_first':
    input_shape = (1, img_rows, img_cols)
else:
    input_shape = (img_rows, img_cols, 1)

In [26]:
## To simplify things, write a function to include all the training steps
## As input, function takes a model, training set, test set, and the number of classes
## Inside the model object will be the state about which layers we are freezing and which we are training

def train_model(model, train, test, num_classes):
    x_train = train[0].reshape((train[0].shape[0],) + input_shape)
    x_test = test[0].reshape((test[0].shape[0],) + input_shape)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')

    # convert class vectors to binary class matrices
    y_train = keras.utils.to_categorical(train[1], num_classes)
    y_test = keras.utils.to_categorical(test[1], num_classes)

    model.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])

    t = now()
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(x_test, y_test))
    print('Training time: %s' % (now() - t))

    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test score:', score[0])
    print('Test accuracy:', score[1])

In [27]:
# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# create two datasets: one with digits below 5 and one with 5 and above
x_train_lt5 = x_train[y_train < 5]
y_train_lt5 = y_train[y_train < 5]
x_test_lt5 = x_test[y_test < 5]
y_test_lt5 = y_test[y_test < 5]

x_train_gte5 = x_train[y_train >= 5]
y_train_gte5 = y_train[y_train >= 5] - 5
x_test_gte5 = x_test[y_test >= 5]
y_test_gte5 = y_test[y_test >= 5] - 5

In [28]:
# Define the "feature" layers.  These are the early layers that we expect will "transfer"
# to a new problem.  We will freeze these layers during the fine-tuning process

feature_layers = [
    Conv2D(filters, kernel_size,
           padding='valid',
           input_shape=input_shape),
    Activation('relu'),
    Conv2D(filters, kernel_size),
    Activation('relu'),
    MaxPooling2D(pool_size=pool_size),
    Dropout(0.25),
    Flatten(),
]

In [29]:
# Define the "classification" layers.  These are the later layers that predict the specific classes from the features
# learned by the feature layers.  This is the part of the model that needs to be re-trained for a new problem

classification_layers = [
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(num_classes),
    Activation('softmax')
]

In [30]:
# We create our model by combining the two sets of layers as follows
model = Sequential(feature_layers + classification_layers)

In [31]:
# Let's take a look
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 activation_8 (Activation)   (None, 26, 26, 32)        0         
                                                                 
 conv2d_5 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 activation_9 (Activation)   (None, 24, 24, 32)        0         
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 12, 12, 32)       0         
 2D)                                                             
                                                                 
 dropout_4 (Dropout)         (None, 12, 12, 32)        0         
                                                      

In [32]:
# Now, let's train our model on the digits 5,6,7,8,9

train_model(model,
            (x_train_gte5, y_train_gte5),
            (x_test_gte5, y_test_gte5), num_classes)

x_train shape: (29404, 28, 28, 1)
29404 train samples
4861 test samples
Epoch 1/5
230/230 [==============================] - 25s 99ms/step - loss: 1.6087 - accuracy: 0.2103 - val_loss: 1.5915 - val_accuracy: 0.2615
Epoch 2/5
230/230 [==============================] - 24s 103ms/step - loss: 1.5893 - accuracy: 0.2558 - val_loss: 1.5701 - val_accuracy: 0.3170
Epoch 3/5
230/230 [==============================] - 24s 102ms/step - loss: 1.5690 - accuracy: 0.3040 - val_loss: 1.5471 - val_accuracy: 0.4059
Epoch 4/5
230/230 [==============================] - 23s 98ms/step - loss: 1.5462 - accuracy: 0.3598 - val_loss: 1.5219 - val_accuracy: 0.5087
Epoch 5/5
230/230 [==============================] - 23s 100ms/step - loss: 1.5230 - accuracy: 0.4075 - val_loss: 1.4942 - val_accuracy: 0.6036
Training time: 0:01:58.322510
Test score: 1.4941926002502441
Test accuracy: 0.6035795211791992


### Freezing Layers
Keras allows layers to be "frozen" during the training process.  That is, some layers would have their weights updated during the training process, while others would not.  This is a core part of transfer learning, the ability to train just the last one or several layers.

Note also, that a lot of the training time is spent "back-propagating" the gradients back to the first layer.  Therefore, if we only need to compute the gradients back a small number of layers, the training time is much quicker per iteration.  This is in addition to the savings gained by being able to train on a smaller data set.


In [33]:
# Freeze only the feature layers
for l in feature_layers:
    l.trainable = False

Observe below the differences between the number of *total params*, *trainable params*, and *non-trainable params*.


In [34]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 activation_8 (Activation)   (None, 26, 26, 32)        0         
                                                                 
 conv2d_5 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 activation_9 (Activation)   (None, 24, 24, 32)        0         
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 12, 12, 32)       0         
 2D)                                                             
                                                                 
 dropout_4 (Dropout)         (None, 12, 12, 32)        0         
                                                      

In [35]:
train_model(model,
            (x_train_lt5, y_train_lt5),
            (x_test_lt5, y_test_lt5), num_classes)

x_train shape: (30596, 28, 28, 1)
30596 train samples
5139 test samples
Epoch 1/5
240/240 [==============================] - 10s 36ms/step - loss: 1.5767 - accuracy: 0.3050 - val_loss: 1.5516 - val_accuracy: 0.4147
Epoch 2/5
240/240 [==============================] - 8s 34ms/step - loss: 1.5398 - accuracy: 0.3666 - val_loss: 1.5121 - val_accuracy: 0.5085
Epoch 3/5
240/240 [==============================] - 8s 34ms/step - loss: 1.5041 - accuracy: 0.4272 - val_loss: 1.4731 - val_accuracy: 0.6098
Epoch 4/5
240/240 [==============================] - 9s 39ms/step - loss: 1.4684 - accuracy: 0.4874 - val_loss: 1.4352 - val_accuracy: 0.6826
Epoch 5/5
240/240 [==============================] - 8s 35ms/step - loss: 1.4336 - accuracy: 0.5444 - val_loss: 1.3978 - val_accuracy: 0.7365
Training time: 0:00:43.612068
Test score: 1.3978396654129028
Test accuracy: 0.7365246415138245


Note that after a single epoch, we are already achieving results on classifying 0-4 that are comparable to those achieved on 5-9 after 5 full epochs.  This despite the fact the we are only "fine-tuning" the last layer of the network, and all the early layers have never seen what the digits 0-4 look like.

Also, note that even though nearly all (590K/600K) of the *parameters* were trainable, the training time per epoch was still much reduced.  This is because the unfrozen part of the network was very shallow, making backpropagation faster. 


## Exercise
- Now we will write code to reverse this training process.  That is, train on the digits 0-4, then finetune only the last layers on the digits 5-9.


In [36]:
# Create layers and define the model as above
feature_layers2 = [
    Conv2D(filters, kernel_size,
           padding='valid',
           input_shape=input_shape),
    Activation('relu'),
    Conv2D(filters, kernel_size),
    Activation('relu'),
    MaxPooling2D(pool_size=pool_size),
    Dropout(0.25),
    Flatten(),
]

classification_layers2 = [
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(num_classes),
    Activation('softmax')
]
model2 = Sequential(feature_layers2 + classification_layers2)
model2.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 activation_12 (Activation)  (None, 26, 26, 32)        0         
                                                                 
 conv2d_7 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 activation_13 (Activation)  (None, 24, 24, 32)        0         
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 12, 12, 32)       0         
 2D)                                                             
                                                                 
 dropout_6 (Dropout)         (None, 12, 12, 32)        0         
                                                      

In [37]:
# Now, let's train our model on the digits 0,1,2,3,4
train_model(model2,
            (x_train_lt5, y_train_lt5),
            (x_test_lt5, y_test_lt5), num_classes)

x_train shape: (30596, 28, 28, 1)
30596 train samples
5139 test samples
Epoch 1/5
240/240 [==============================] - 30s 120ms/step - loss: 1.5913 - accuracy: 0.2777 - val_loss: 1.5731 - val_accuracy: 0.4532
Epoch 2/5
240/240 [==============================] - 24s 102ms/step - loss: 1.5631 - accuracy: 0.3722 - val_loss: 1.5404 - val_accuracy: 0.6511
Epoch 3/5
240/240 [==============================] - 24s 99ms/step - loss: 1.5305 - accuracy: 0.4674 - val_loss: 1.5016 - val_accuracy: 0.7902
Epoch 4/5
240/240 [==============================] - 24s 99ms/step - loss: 1.4924 - accuracy: 0.5529 - val_loss: 1.4550 - val_accuracy: 0.8653
Epoch 5/5
240/240 [==============================] - 26s 107ms/step - loss: 1.4452 - accuracy: 0.6266 - val_loss: 1.3983 - val_accuracy: 0.8978
Training time: 0:02:07.424502
Test score: 1.3982841968536377
Test accuracy: 0.8978400230407715


In [38]:
#Freeze layers
for l in feature_layers2:
    l.trainable = False

In [39]:
model2.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 activation_12 (Activation)  (None, 26, 26, 32)        0         
                                                                 
 conv2d_7 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 activation_13 (Activation)  (None, 24, 24, 32)        0         
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 12, 12, 32)       0         
 2D)                                                             
                                                                 
 dropout_6 (Dropout)         (None, 12, 12, 32)        0         
                                                      

In [40]:
train_model(model2,
            (x_train_gte5, y_train_gte5),
            (x_test_gte5, y_test_gte5), num_classes)

x_train shape: (29404, 28, 28, 1)
29404 train samples
4861 test samples
Epoch 1/5
230/230 [==============================] - 9s 36ms/step - loss: 1.6180 - accuracy: 0.2439 - val_loss: 1.5927 - val_accuracy: 0.3693
Epoch 2/5
230/230 [==============================] - 9s 37ms/step - loss: 1.5891 - accuracy: 0.2822 - val_loss: 1.5611 - val_accuracy: 0.4079
Epoch 3/5
230/230 [==============================] - 8s 35ms/step - loss: 1.5609 - accuracy: 0.3282 - val_loss: 1.5299 - val_accuracy: 0.4781
Epoch 4/5
230/230 [==============================] - 8s 35ms/step - loss: 1.5334 - accuracy: 0.3815 - val_loss: 1.4994 - val_accuracy: 0.5869
Epoch 5/5
230/230 [==============================] - 8s 36ms/step - loss: 1.5043 - accuracy: 0.4428 - val_loss: 1.4696 - val_accuracy: 0.6731
Training time: 0:00:42.160421
Test score: 1.4695541858673096
Test accuracy: 0.6731125116348267


---
### Machine Learning Foundation (C) 2020 IBM Corporation
